In [1]:
# multiclass text classification
import pandas as pd
df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [2]:
# one hot encoding
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns=['genre'])
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [3]:
# split attribute and label
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [4]:
# split data into train and test
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [5]:
# implement tokenizer and pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih)
tokenizer.fit_on_texts(sinopsis_test)

sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

In [6]:
# create model
import tensorflow as tf
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
# train model
num_epochs = 30
model.fit(
    padded_latih,
    label_latih,
    epochs=num_epochs,
    validation_data=(padded_test, label_test),
    verbose=2
)

Epoch 1/30
26/26 - 14s - loss: 1.6100 - accuracy: 0.1654 - val_loss: 1.6101 - val_accuracy: 0.1940 - 14s/epoch - 529ms/step
Epoch 2/30
26/26 - 5s - loss: 1.6082 - accuracy: 0.2525 - val_loss: 1.6111 - val_accuracy: 0.2040 - 5s/epoch - 192ms/step
Epoch 3/30
26/26 - 6s - loss: 1.5923 - accuracy: 0.3520 - val_loss: 1.5945 - val_accuracy: 0.2338 - 6s/epoch - 229ms/step
Epoch 4/30
26/26 - 5s - loss: 1.4616 - accuracy: 0.4502 - val_loss: 1.5822 - val_accuracy: 0.2289 - 5s/epoch - 179ms/step
Epoch 5/30
26/26 - 5s - loss: 0.9787 - accuracy: 0.6592 - val_loss: 1.9668 - val_accuracy: 0.2886 - 5s/epoch - 184ms/step
Epoch 6/30
26/26 - 6s - loss: 0.5334 - accuracy: 0.8072 - val_loss: 2.4300 - val_accuracy: 0.2836 - 6s/epoch - 240ms/step
Epoch 7/30
26/26 - 4s - loss: 0.2869 - accuracy: 0.8968 - val_loss: 2.7355 - val_accuracy: 0.3085 - 4s/epoch - 156ms/step
Epoch 8/30
26/26 - 6s - loss: 0.1849 - accuracy: 0.9415 - val_loss: 3.3316 - val_accuracy: 0.2687 - 6s/epoch - 243ms/step
Epoch 9/30
26/26 - 3s 